In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import cv2

In [ ]:
device_numbers = ['21187339', '21187335', '21174907']  # 21187339 is master
get_device_text_path = lambda device_number: f"headtracking_2023-05-03_09-26-29_{device_number}.txt"
get_device_video_path = lambda device: f"headtracking_2023-05-03_09-26-29_{device}-0000.avi"

def get_data(device):
    with open(get_device_text_path(device)) as fh:
        def _process_line(line):
            a, b = line.strip().split(',')
            return int(a), float(b)
        return dict(_process_line(line) for line in fh)

In [ ]:
data = [get_data(device) for device in device_numbers]
df = pd.DataFrame(data).T

In [ ]:
plt.plot(df.loc[130:280], label=[device_numbers[i] for i in df.columns])
plt.ylabel('time [ms]')
plt.xlabel('frame')
plt.title('frame drops')
plt.legend()

In [ ]:
# minima = ((df - df.loc[11341, 0])**2).idxmin()
minima = ((df - df.loc[8816, 0])**2).idxmin()
minima = ((df - df.loc[51693, 0])**2).idxmin()

print(minima)
times = [df.loc[minima[i], i] for i in range(3)]
print(times)

fig, ax = plt.subplots(3)
fig.set_size_inches(30,20)
for i in range(3):
    cap = cv2.VideoCapture(get_device_video_path(device_numbers[i]))
    frame_number = minima[i]
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number-1)
    res, frame = cap.read()
    ax[i].set_title(f'frame: {frame_number} time: {times[i]}')
    ax[i].imshow(frame)
    ax[i].axis('off')